In [11]:
import os

In [12]:
hello_world_script_file = os.path.join(os.path.pardir, 'src', 'models', 'hello_world_api.py')

In [13]:
%%writefile $hello_world_script_file
from flask import Flask, request
app = Flask(__name__)
@app.route('/api', methods = {'POST'})
def say_hello():
    data = requests.get_json(force=True)
    name = data['name']
    return "Hello {0}".format(name)
if __name__ == "__main__":
    app.run(port=1001, debug=True)

Overwriting ..\src\models\hello_world_api.py


In [14]:
import json
import requests

In [17]:
url = 'http://127.0.0.1;1001/api'
data = json.dumps({'name':'goro'})
r = requests.post(url, data)

ConnectionError: HTTPConnectionPool(host='127.0.0.1;1001', port=80): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000240A4C4E1F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## Machine Learning API Using Flask

In [18]:
ml_api_script_file = os.path.join(os.path.pardir, 'src', 'models', 'ml_api.py')

In [19]:
%%writefile $ml_api_script_file
from flask import Flask, request
import numpy as np
import pandas as pd
import json
import pickle
import os

app = Flask(__name__)
# Load model and scaler files
model_path = os.path.join(os.path.pardir, 'models')
model_filepath = os.path.join(model_path, 'lr_model.pkl')
scaler_filepath = os.path.join(model_path, 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath))
model = pickle.load(open(model_filepath))

# columns
columns = ['Age', 'Fare', 'Family_size', 'Is_mother', 'Is_male',
       'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G',
       'Deck_Z', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Lady',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Sir', 'Fare_Bin_Very_Low', 'Fare_Bin_Low', 'Fare_Bin_High',
       'Fare_Bin_Very_High', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Is_adult']

@pp.route('/api', methods=['POST'])
def make_predictions():
    # read json object and convert to json string
    data = json.dumps(requests.get_json(force=True))
    # create pandas dataframe using json string
    df = pd.read_json(data)
    # extract passengerids
    passenger_ids = df.PassengerId.ravel()
    # actual survivedd values
    actuals = df.Survived.ravel()
    # exctract required columns and convert to matrix
    X = df[columns].to_numpy().astype('float')
    # transform the input
    X_scaled = scaler.transform(X)
    # make predictions
    predictions = model.predict(X_scaled)
    # create response dataframe
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predicted': predictions, 'Actuals': actuals})
    # return json
    return df_response.to_json()
if __name__ == "__main__":
    # host flask app at port 10001
    app.run(port=10001, debug=True)
    

Writing ..\src\models\ml_api.py


## Invoking API using requests

In [20]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [21]:
survived_passengers = train_df[train_df.Survived == 1][:5]

In [23]:
import requests
def make_api_request(data):
    # url for api
    url = 'http://127.0.0.1:10001/api'
    # make post request
    r = requests.post(url, data)
    return r.json()

In [24]:
#make_api_request(survived_passengers.to_json())